In [2]:
import os
import json
import pandas as pd
import ujson as json
import time
import ijson
from mpi4py import MPI
import re

In [3]:
comm = MPI.COMM_WORLD
# rank = comm.Get_rank()
# size = comm.Get_size()
f=open('mnt/ext100/twitter-huge.json', encoding='utf-8')
g=open('twitter-place-data.json','w',encoding='utf-8')
g.write('[\n')
f.seek(0,os.SEEK_END)
size=f.tell()
print(size)
f.seek(0)
metadata=f.readline()
# print(metadata)
nrows=int(metadata.split(':')[1].split(',')[0])
print(nrows)
nrows=nrows
# startrow=rank*chunkrows
# endrow=(rank+1)*chunkrows
nrows=nrows-1
try:
    for i in range(nrows):
        data=f.readline()
        try:
            data=json.loads(data[:-2])
        except:
            print(f'Error in row and location:{i} {f.tell()}')
            if f.tell()>=size:
                break
        # print(data['doc'].keys())
        else:
            if 'includes' in data['doc'].keys():
                last=data
                # pass
                # print(data)
                g.write(json.dumps(data)+',\n')
    # for key,value in data.items():
    #     print(key, value)
except:
    print(f'File at location {f.tell()}')
g.write(json.dumps(last)+'\n')
g.write(']')
g.close()

63201091459
166476331


In [32]:
f.seek(0,os.SEEK_END)
size=f.tell()
print(size)
f.seek(size-10000)
while(f.tell()<size):
    data=f.readline()
    print(data)
# f.readline()
63201091459/166476331

63201091459
nburyWeather @baxlex @perthovalman @ManjitK6987 @ImTheOnlyAstro @RKMac65 @TrixieBelden_ @drstip @2017Ferret @paulmp @speedbird020 @Richard_Kreider @Sparrow_65 @sivideoaviation @Barnsy_Lisa @BigV2011WCE @wiccewicker @Rob_lebob @TheWAWG @aussie_robbob @WendyBirdOZ @N8aviation @perthobs #Scoot Have A New Pokémon Themed Special Livery #Boeing 787 (9V-OJJ) The Plane Was Painted In The New Livery In July. It Last Visited #Perth On The 27th Of June And Hasn't Flown Since The 28th Of June But Should Be Back In Service Soon. #AvGeek #PerthAirport https://t.co/XFQEBuEdeq","sentiment":0},"matching_rules":[{"id":"1557026228036861952","tag":""}]}},

{"id":"1557348633275953152","key":[2022,8,10,"988404415789846530","986092014","1557348633275953152"],"value":{"tags":"Scoot|Boeing|AvGeek|PerthAirport|PlaneSpotting","tokens":"Here|The|New|787|OJJ"},"doc":{"_id":"1557348633275953152","_rev":"1-7ae4b92ab7adffead46527d169131d8d","data":{"author_id":"986092014","context_annotations":[{"domain":

379.6401030666636

In [5]:
h=open('twitter-place-data.json','r', encoding='utf-8')
h.seek(0,os.SEEK_END)
size=h.tell()
print(size)
h.seek(size-1000)
data=h.read()
print(data)
# obj=ijson.items(h,'item')
# i=0
# for ob in obj:
#     i+=1
# print(i)


4520394668
 DiCaprio"},"entity":{"id":"1110491889365454848","name":"Taika Waititi"}},{"domain":{"id":"131","name":"Unified Twitter Taxonomy","description":"A taxonomy view into the Semantic Core knowledge graph"},"entity":{"id":"1110491889365454848","name":"Taika Waititi"}}],"conversation_id":"291489018783363072","created_at":"2022-08-10T19:28:52.000Z","entities":{"mentions":[{"start":0,"end":9,"username":"bluboy43","id":"1297204830231040005"},{"start":10,"end":23,"username":"TaikaWaititi","id":"95610041"}]},"geo":{"place_id":"004ec16c62325149"},"lang":"en","public_metrics":{"retweet_count":0,"reply_count":0,"like_count":0,"quote_count":0},"text":"@bluboy43 @TaikaWaititi People will still like it. You don't have to.","sentiment":0.18181818181818182},"includes":{"places":[{"full_name":"Brisbane, Queensland","geo":{"type":"Feature","bbox":[152.668522848,-27.767440994,153.31787024,-26.996844991],"properties":{}},"id":"004ec16c62325149"}]},"matching_rules":[{"id":"1557388137877635072","tag